# Data Loader, Model, Trainning
# Dependencies

In [1]:
%load_ext autoreload

In [2]:
autoreload 2

In [4]:
from dataset import *
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from transformers import BertModel

I0708 02:29:03.224430 140051427272512 file_utils.py:39] PyTorch version 1.5.1+cu101 available.
I0708 02:29:04.914098 140051427272512 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


# Mode

In [4]:
TEST = True
LOG = True

# Hyper Params

In [44]:
BATCH_SIZE = 64
n_class = 2
PRETRAINED_MODEL_NAME = 'bert-base-chinese'

# File Path ...

In [45]:
train_data_path = './FGC_release_1.7.13/FGC_release_all_dev.json'

In [46]:
#default tolkenizer is used
train_set = FGC_Dataset(train_data_path, "train") 

In [47]:
if TEST or LOG:
    print(train_set.tokenizer.convert_ids_to_tokens(train_set[0][0]), "\n",
    train_set[0][1],"\n",
    train_set[0][2])

['[CLS]', '苏', '东', '坡', '在', '中', '国', '历', '史', '上', '，', '是', '哪', '一', '个', '朝', '代', '的', '人', '？', '[SEP]', '苏', '轼', '（', '103', '##7', '年', '1', '月', '8', '日', '－', '110', '##1', '年', '8', '月', '24', '日', '）', '，', '[SEP]'] 
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) 
 tensor(True)


In [48]:
# DataLoader
# collate_fn : list of instances to minibatch
trainloader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [49]:
if TEST:
    data = next(iter(trainloader))

    tokens_tensors, segments_tensors, masks_tensors, label_ids = data

    print(f"""
    tokens_tensors.shape   = {tokens_tensors.shape} 
    {tokens_tensors}
    ------------------------
    segments_tensors.shape = {segments_tensors.shape}
    {segments_tensors}
    ------------------------
    masks_tensors.shape    = {masks_tensors.shape}
    {masks_tensors}
    ------------------------
    label_ids.shape        = {label_ids.shape}
    {label_ids}
    """)


    tokens_tensors.shape   = torch.Size([64, 55]) 
    tensor([[ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        ...,
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0],
        [ 101, 5722,  691,  ...,    0,    0,    0]])
    ------------------------
    segments_tensors.shape = torch.Size([64, 76])
    tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
    ------------------------
    masks_tensors.shape    = torch.Size([64, 55])
    tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
    ------------

In [50]:
# calc pos weight for BCE
total = 0
true_cnt = 0
for instance in train_set:
    if(instance[-1] == True):
        true_cnt += 1
    total += 1
print(true_cnt)
print(total)
print(torch.tensor([total/true_cnt, total/(total-true_cnt)]))
# to increase the value of recall in the model's criterion
pos_weight = print(torch.tensor([(total-true_cnt)/true_cnt, 1]))
print(pos_weight)
# no need to applied pos_weight = torch.tensor([total/true_cnt, total/(1-true_cnt)])?

547
8843
tensor([16.1664,  1.0659])
tensor([15.1664,  1.0000])
None


In [52]:
bert_model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)

I0707 03:40:08.506779 139996915132224 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
I0707 03:40:08.509397 139996915132224 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_he

# Model

In [53]:
class BertSERModel(nn.Module):
    """
    baseline
    naive bert by NSP stype + linear classifier applied on [CLS] last hidden
    """
    
    def __init__(self, bert_encoder=None):
        super(BertSERModel, self).__init__()
        if bert_encoder == None or not isinstance(bert_encoder, BertModel):
            print("unkown bert model choice, init with PRETRAINED_MODEL_NAME")
            self.bert_encoder = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
        self.bert_encoder = bert_encoder
        self.dropout = nn.Dropout(p=bert_encoder.config.hidden_dropout_prob)
        self.classifier = nn.Linear(bert_encoder.config.hidden_size, 1)
        # critrion add positive weight
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    def forward_nn(self, batch):
        """
        batch[0] = input
        batch[1] = token_type_ids (which sent)
        batch[2] = mask for padding
        """
        # the _ here is the last hidden states
        # q_poolout is a 768-d vector of [CLS]
        _, q_poolout = self.bert_encoder(batch[0],
                                         token_type_ids=batch[1],
                                         attention_mask=batch[2])
        # q_poolout = self.dropout(q_poolout), MT Wu : no dropout better, without 
        logits = self.classifier(q_poolout)
        # can apply nn.module.Sigmoid here to convert to p-distribution
        # score is indeed better (and more stable)
        logits = logits.squeeze(-1)
        return logits
    
    # the nn.Module method
    def forward(self, batch):
        logits = self.forward_nn(batch)
        loss = self.criterion(logits, batch['label'])
        return loss
    
    # return sigmoded score
    def _predict(self, batch):
        logits = self.forward_nn(batch)
        scores = torch.sigmoid(logits)
        scores = scores.cpu().numpy().tolist()
        return scores
    
    # return result with assigned threshold, default = 0.5
    def predict_fgc(self, q_batch, threshold=0.5):
        scores = self._predict(q_batch)

        max_i = 0
        max_score = 0
        sp = []
        for i, score in enumerate(scores):
            if score > max_score:
                max_i = i
                max_score = score
            if score >= threshold:
                sp.append(i)

        if not sp:
            sp.append(max_i)

        return {'sp': sp, 'sp_scores': scores}

In [54]:
base_line_model = BertSERModel(bert_model)
None

In [56]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(base_line_model)
clf_params = get_learnable_params(base_line_model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102268417
線性分類器的參數量：769



hi
CPU times: user 194 µs, sys: 39 µs, total: 233 µs
Wall time: 165 µs
